In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
import torch.nn.functional as F
from torchvision import transforms

from PIL import Image
import matplotlib.pyplot as plt

In [2]:
class Params:
  class_num = 10
  batch_size = 1
  learning_rate = 0.001
  dropout = 0.2
  momentum = 0.9
  epochs = 10


In [3]:
base_path = "../leapGestRecog/"
people = ["00", "01", "02", "03", "04", "05", "06", "07", "08", "09"]
gestures = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10"]
gestures_map = {
    1: "palm", 
    2: "l", 
    3: "fist", 
    4: "fist_moved",
    5: "thumb", 
    6: "index", 
    7: "ok", 
    8: "palm_moved", 
    9: "c",
    10: "down"
}

class GestureDataset(Dataset):
  def __init__(self, transform = None, target_transform = None):
    img_labels = []
    for person in range(10):
      for label in range(1, 11):
        path = "%s/%s_%s/" % (str(person).zfill(2), str(label).zfill(2), gestures_map[label])
        for id in range(1, 201):
          filename = "frame_%s_%s_%s.png" % (str(person).zfill(2), str(label).zfill(2), str(id).zfill(4))
          img_labels.append((base_path + path + filename, label))
          
    self.img_labels = img_labels 
    self.transform = transform
    self.target_transform = target_transform
  def __getitem__(self, index):
    path, label = self.img_labels[index]
    img = Image.open(path).convert('L') # convert to black and white
    if self.transform is not None:
      img = self.transform(img)
    return img, label - 1
  def __len__(self):
    return len(self.img_labels)

In [4]:
gesture_dataset = GestureDataset(transform=transforms.ToTensor())

train_size = int(20000*0.8)
test_size = int(20000*0.2)

train_dataset, test_dataset = torch.utils.data.random_split(gesture_dataset, [train_size, test_size])

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

class Model(nn.Module):
  def __init__(self, dropout):
    super(Model, self).__init__()
    self.conv1 = nn.Conv2d(1,6, kernel_size=5, stride=1, padding=0)
    self.max_pool1 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
    self.conv2 = nn.Conv2d(6, 16, kernel_size=5, padding=0)
    self.conv2_drop = nn.Dropout2d(p=dropout)
    self.max_pool2 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

    self.fc1 = nn.Linear(143184, 512) # number channels * width * height
    self.fc2 = nn.Linear(512, 10)
    self.fc1_drop = nn.Dropout(p=dropout)

  def forward(self, x):
    # print(x.size(0))
    # print(x.shape)
    x = torch.relu(self.max_pool1(self.conv1(x)))
    # print(x.shape)
    x = torch.relu(self.max_pool2(self.conv2_drop(self.conv2(x))))
    
    # flatten over channel, height and width
    # print(x.shape)
    x = x.view(x.size(0), x.size(1) * x.size(2) * x.size(3))
    
    x = torch.relu(self.fc1_drop(self.fc1(x)))
    # x = torch.softmax(self.fc2(x), dim=-1)
    x = torch.relu(self.fc2(x))
    return x


model = Model(dropout=Params.dropout).to(device)

Using cuda device


In [6]:

history = {
  "loss": [],
  "acc": [],
  "loss_iter": [],
  "val_loss": [],
  "val_acc": []
}

def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.train()
  epoch_loss, epoch_acc = 0, 0
  for batch, (X, y) in enumerate(dataloader):
    # y = F.one_hot(y, num_classes=10).float()
    X, y = X.to(device), y.to(device)

    # Compute prediction error
    pred = model(X)
    # print(pred.argmax(1))
    loss = loss_fn(pred, y)
  
    history["loss_iter"].append(loss.item())
    epoch_loss += loss.item()
    epoch_acc += (pred.argmax(1) == y).type(torch.float).sum().item()

    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
  
  epoch_loss /= num_batches
  epoch_acc /= size
  print(epoch_loss)
  print(epoch_acc)
  history["loss"].append(epoch_loss)
  history["acc"].append(epoch_acc)

def validate(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  val_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      val_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  val_loss /= num_batches
  correct /= size
  history["val_loss"].append(val_loss)
  history["val_acc"].append(correct)
  print(f"Validation: \n Accuracy: {(100*correct):>0.2f}%, Avg loss: {val_loss:>8f} \n")

def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.2f}%, Avg loss: {test_loss:>8f} \n")

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=Params.learning_rate, momentum=Params.momentum)


for t in range(Params.epochs):
  print(f"Epoch {t+1}\n-------------------------------")
  train_size = int(20000*0.7)
  val_size = int(20000*0.1)
  
  _, val_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

  train_dataloader = DataLoader(train_dataset, batch_size=Params.batch_size, shuffle=True, num_workers=0, pin_memory=True)
  val_dataloader = DataLoader(val_dataset, batch_size=Params.batch_size, shuffle=True, num_workers=0, pin_memory=True)
  train(train_dataloader, model, loss_fn, optimizer)
  validate(val_dataloader, model, loss_fn)
print("Done!")


Epoch 1
-------------------------------
loss: 2.297423  [    0/16000]


RuntimeError: CUDA out of memory. Tried to allocate 280.00 MiB (GPU 0; 2.00 GiB total capacity; 851.07 MiB already allocated; 109.19 MiB free; 864.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
test_dataloader = DataLoader(test_dataset, batch_size=Params.batch_size, shuffle=True, num_workers=0, pin_memory=True)
test(test_dataloader, model, loss_fn)

In [ ]:
# 保存
torch.save(model, 'model.pkl')
# 加载
# model = torch.load('\model.pkl')

In [ ]:
# Display image and label.
train_features, train_labels = next(iter(train_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")
img = train_features[0].squeeze()
label = train_labels[0]
plt.imshow(img)
plt.show()
print(f"Label: {label}")

In [ ]:
fig = plt.figure() # 新建一张图
# plt.plot(history["loss"], label='training loss')
plt.plot(history['val_loss'], label="val_loss")
plt.title('loss')
# plt.ylabel(attr)
plt.xlabel('epoch')
plt.legend()
plt.show()

fig = plt.figure()
plt.plot(history['val_acc'], label="val_acc")
plt.title('acc')
# plt.ylabel(attr)
plt.xlabel('epoch')
plt.legend()
plt.show()

In [ ]:
# import torch
# torch.cuda.empty_cache()